# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Automated-Model-Selection" data-toc-modified-id="Automated-Model-Selection-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Automated Model Selection</a></div><div class="lev2 toc-item"><a href="#Dataset" data-toc-modified-id="Dataset-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Dataset</a></div><div class="lev2 toc-item"><a href="#Grid-testing" data-toc-modified-id="Grid-testing-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Grid testing</a></div><div class="lev3 toc-item"><a href="#Dataset-Iris" data-toc-modified-id="Dataset-Iris-121"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Dataset Iris</a></div><div class="lev4 toc-item"><a href="#Trochę-czystsza-prezentacja" data-toc-modified-id="Trochę-czystsza-prezentacja-1211"><span class="toc-item-num">1.2.1.1&nbsp;&nbsp;</span>Trochę czystsza prezentacja</a></div><div class="lev3 toc-item"><a href="#Dataset-Digits" data-toc-modified-id="Dataset-Digits-122"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Dataset Digits</a></div><div class="lev2 toc-item"><a href="#Testowanie-kilku-klasyfikatorów" data-toc-modified-id="Testowanie-kilku-klasyfikatorów-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Testowanie kilku klasyfikatorów</a></div><div class="lev3 toc-item"><a href="#Dataset-Iris" data-toc-modified-id="Dataset-Iris-131"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Dataset Iris</a></div><div class="lev4 toc-item"><a href="#Parametery-najlepszego-klasyfikatora" data-toc-modified-id="Parametery-najlepszego-klasyfikatora-1311"><span class="toc-item-num">1.3.1.1&nbsp;&nbsp;</span>Parametery najlepszego klasyfikatora</a></div><div class="lev3 toc-item"><a href="#Dataset-Digits" data-toc-modified-id="Dataset-Digits-132"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Dataset Digits</a></div><div class="lev4 toc-item"><a href="#Parametry-najlepszego-klasyfikatora" data-toc-modified-id="Parametry-najlepszego-klasyfikatora-1321"><span class="toc-item-num">1.3.2.1&nbsp;&nbsp;</span>Parametry najlepszego klasyfikatora</a></div><div class="lev3 toc-item"><a href="#Porównanie-parametrów-klasyfikatorów-dla-obu-zestawów-danych" data-toc-modified-id="Porównanie-parametrów-klasyfikatorów-dla-obu-zestawów-danych-133"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Porównanie parametrów klasyfikatorów dla obu zestawów danych</a></div><div class="lev3 toc-item"><a href="#Classification-report" data-toc-modified-id="Classification-report-134"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>Classification report</a></div><div class="lev2 toc-item"><a href="#Przykładowe-wykorzystanie-wyuczonego-klasyfikatora" data-toc-modified-id="Przykładowe-wykorzystanie-wyuczonego-klasyfikatora-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Przykładowe wykorzystanie wyuczonego klasyfikatora</a></div>

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import randint
from operator import itemgetter
import sklearn.datasets as datasets
from sklearn import svm, grid_search, datasets, cross_validation, metrics, ensemble, neighbors, linear_model
from datetime import datetime

# Automated Model Selection

## Dataset

In [2]:
iris_dataset = datasets.load_iris()
digits_dataset = datasets.load_digits(10)

In [3]:
print(iris_dataset['DESCR'])

Iris Plants Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)
    ============== ==== ==== ======= ===== ====================

    :Missing Attribute Values: None
  

## Grid testing

Inicjalizujemy klasyfikator SVM  

**SVM** - Support Vector Machine:
- **SVC** - Support Vector Classification
- **SVR** - Support Vector Regressor
    
[scikit-learn docs](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

In [4]:
svc = svm.SVC()

Słownik parametrów do przetestowania. Grid Search przyjmuję słownik lub listę słowników parametrów, które należy przekazać do konstruktora klasyfikatora. Z każdego słownika tworzy macierz kombinacji tych parametrów.

In [5]:
parameters = [{'kernel':['rbf'], 'C':[1e-4, 1e-3, 1e-2, 1e0, 1e1, 1e2, 1e3]},
              {'kernel':['poly'], 'C':[1e-4, 1e-3, 1e-2, 1e0, 1e1, 1e2, 1e3], 'degree':[2,3]},
             {'kernel':['linear'], 'C':[1e-4, 1e-3, 1e-2, 1e0, 1e1, 1e2, 1e3]}]

### Dataset Iris

Sprawdźmy, które parametry najlepiej się sprawdzają w połączeniu klasyfikatora SVM i zestawu Iris

Po wywołaniu metody **fit(X,y)** na obiekcie **GridSearchCV**,  wykonywana jest kroswalidacja na instancjach klasyfikatora SVM z różnymi kombinacjami parametrów. Po otrzymaniu wyników z każdej kroswalidacji, wybierany jest najlepszy zestaw parametrów.

Walidacja krzyżowa zwraca wyniki otrzymane z domyślnej metody **score(X,y)** zaimplementowanej w klasyfikatorze. Możemy przekazać inny *scorer*, na podstawie którego ma być robiona ocena  
[Model evaluation - scikit-learn doc](http://scikit-learn.org/stable/modules/model_evaluation.html)   
___

In [6]:
gs = grid_search.GridSearchCV(svc, parameters, n_jobs=4)
gs.fit(iris_dataset['data'], iris_dataset['target'])

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=4,
       param_grid=[{'kernel': ['rbf'], 'C': [0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0]}, {'degree': [2, 3], 'kernel': ['poly'], 'C': [0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0]}, {'kernel': ['linear'], 'C': [0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

Zestaw najlepszych parametrów

In [7]:
gs.best_params_

{'C': 10.0, 'kernel': 'rbf'}

Wyniki dla wszystkich zestawów parametrów

In [8]:
gs.grid_scores_

[mean: 0.93333, std: 0.03277, params: {'kernel': 'rbf', 'C': 0.0001},
 mean: 0.93333, std: 0.03277, params: {'kernel': 'rbf', 'C': 0.001},
 mean: 0.93333, std: 0.03277, params: {'kernel': 'rbf', 'C': 0.01},
 mean: 0.97333, std: 0.00897, params: {'kernel': 'rbf', 'C': 1.0},
 mean: 0.98000, std: 0.01601, params: {'kernel': 'rbf', 'C': 10.0},
 mean: 0.96000, std: 0.04217, params: {'kernel': 'rbf', 'C': 100.0},
 mean: 0.95333, std: 0.03669, params: {'kernel': 'rbf', 'C': 1000.0},
 mean: 0.86000, std: 0.00404, params: {'degree': 2, 'kernel': 'poly', 'C': 0.0001},
 mean: 0.87333, std: 0.00809, params: {'degree': 3, 'kernel': 'poly', 'C': 0.0001},
 mean: 0.88667, std: 0.02253, params: {'degree': 2, 'kernel': 'poly', 'C': 0.001},
 mean: 0.96667, std: 0.01820, params: {'degree': 3, 'kernel': 'poly', 'C': 0.001},
 mean: 0.95333, std: 0.02402, params: {'degree': 2, 'kernel': 'poly', 'C': 0.01},
 mean: 0.97333, std: 0.02446, params: {'degree': 3, 'kernel': 'poly', 'C': 0.01},
 mean: 0.96667, std: 

#### Trochę czystsza prezentacja

In [9]:
# source: http://scikit-learn.org/stable/auto_examples/model_selection/randomized_search.html
# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.5f} (std: {1:.5f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [10]:
report(gs.grid_scores_, 5)

Model with rank: 1
Mean validation score: 0.98000 (std: 0.01601)
Parameters: {'kernel': 'rbf', 'C': 10.0}

Model with rank: 2
Mean validation score: 0.98000 (std: 0.01602)
Parameters: {'kernel': 'linear', 'C': 1.0}

Model with rank: 3
Mean validation score: 0.97333 (std: 0.00897)
Parameters: {'kernel': 'rbf', 'C': 1.0}

Model with rank: 4
Mean validation score: 0.97333 (std: 0.02446)
Parameters: {'degree': 3, 'kernel': 'poly', 'C': 0.01}

Model with rank: 5
Mean validation score: 0.97333 (std: 0.03697)
Parameters: {'kernel': 'linear', 'C': 10.0}



### Dataset Digits

Dokładnie to samo dla zestawu danych Digits

In [11]:
gs = grid_search.GridSearchCV(svc, parameters, n_jobs=4)
gs.fit(digits_dataset['data'], digits_dataset['target'])

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=4,
       param_grid=[{'kernel': ['rbf'], 'C': [0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0]}, {'degree': [2, 3], 'kernel': ['poly'], 'C': [0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0]}, {'kernel': ['linear'], 'C': [0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [12]:
report(gs.grid_scores_, 5)

Model with rank: 1
Mean validation score: 0.96049 (std: 0.01269)
Parameters: {'degree': 3, 'kernel': 'poly', 'C': 0.0001}

Model with rank: 2
Mean validation score: 0.96049 (std: 0.01269)
Parameters: {'degree': 3, 'kernel': 'poly', 'C': 0.001}

Model with rank: 3
Mean validation score: 0.96049 (std: 0.01269)
Parameters: {'degree': 3, 'kernel': 'poly', 'C': 0.01}

Model with rank: 4
Mean validation score: 0.96049 (std: 0.01269)
Parameters: {'degree': 3, 'kernel': 'poly', 'C': 1.0}

Model with rank: 5
Mean validation score: 0.96049 (std: 0.01269)
Parameters: {'degree': 3, 'kernel': 'poly', 'C': 10.0}



## Testowanie kilku klasyfikatorów

Wcześniej przetestowaliśmy jeden klasyfikator z różnymi parametrami.  
Dzięki temu dowiedzieliśmy się jaki najlepszy wynik można osiągnąć z danym klasyfikatorem. Sprawdzając w ten sposób kilka różnych klasyfikatorów, bardzo łatwo jest wybrać najlepszy z nich (przynajmniej pod względem danej metody oceniania - _scoring_)

In [13]:
def test_classifiers(classifiers_to_test, X, y):
    """
    Zwróć listę przekazanych klasyfikatorów z najlepszymi wynikami osiągnietymi dla zestawu danych X,y
    
    Args:
        classifiers_to_test: lista klasyfikatorów do sprawdzenia, 
                             składa się z słowników z wartościami: name, classifier, parameters
        X: dane wejściowe do klasyfikacji
        y: klasy odpowiadające danym wejściowym
    """
    classifiers_results = []
    for classifier in classifiers_to_test:  # iterujemy po każdym słowniku z informacjami o klasyfikatorze
        start = datetime.now()
        gs = grid_search.GridSearchCV(classifier['classifier'],  # inicjalizujemy GridSearchCV
                                      classifier['parameters'], 
                                      n_jobs=4,  # GridSearchCV będzie działał na 4 wątkach
                                      scoring='accuracy')  # ustawiamy wszystkim klasyfikatorom na sztywno rodzaj _scoringu_
        gs.fit(X, y)
        classifiers_results.append({'name': classifier['name'], 
                                         'best_score': gs.best_score_, 
                                         'grid_search': gs})  # zapisujemy wyniki

        # na bierząco wyświetlamy wyniki
        print("{0}\nscore: {1}\ntime: {2}\n*=======================*".
              format(classifier['name'], 
                     gs.best_score_, 
                     (datetime.now() - start))) 
        
    return classifiers_results

Tworzymy listę klasyfikatorów z odpowiednimi zestawami parametrów. W celu czytelniejszej prezentacji przekazujemy też nazwę klasyfikatora.

In [14]:
classifiers_to_test = [
    {
        'name': 'SVC',
        'classifier': svm.SVC(),
        'parameters': [{'kernel':['rbf'], 'C':[1e-4, 1e-3, 1e-2, 1e0, 1e1, 1e2, 1e3]},
                       {'kernel':['linear'], 'C':[1e-4, 1e-3, 1e-2, 1e0, 1e1, 1e2, 1e3]}]
    },
    {
        'name': 'LinearSVC',
        'classifier': svm.LinearSVC(),
        'parameters': [{'max_iter':[1e3, 1e4, 1e5], 'C':[1e-4, 1e-3, 1e-2, 1e0, 1e1, 1e2, 1e3]}]
    },
    {
        'name': 'ExtraTreesClassifier',
        'classifier': ensemble.ExtraTreesClassifier(),
        'parameters': [{'n_estimators':[5,10,20,50], 'min_samples_leaf':[1,2,4,8]}]
    },
    {
        'name': 'RandomForestClassifier',
        'classifier': ensemble.RandomForestClassifier(),
        'parameters': [{'n_estimators':[5,10,20,50], 'min_samples_leaf':[1,2,4,8]}]
    },
    {
        'name': 'KNeighborsClassifier',
        'classifier': neighbors.KNeighborsClassifier(),
        'parameters': [{'n_neighbors':[5,10,20], 'p':[1,2]}]
    },
    {
        'name': 'LogisticRegression',
        'classifier': linear_model.LogisticRegression(),
        'parameters': [{'max_iter':[1e3, 1e4, 1e5], 'C':[1e-4, 1e-3, 1e-2, 1e0, 1e1, 1e2, 1e3]}]
    }
]

### Dataset Iris

In [15]:
X, y = iris_dataset['data'], iris_dataset['target']
classifiers_results_iris = test_classifiers(classifiers_to_test, X , y)

SVC
score: 0.98
time: 0:00:00.172051
*=======================*
LinearSVC
score: 0.9733333333333334
time: 0:00:02.309578
*=======================*
ExtraTreesClassifier
score: 0.9666666666666667
time: 0:00:00.768941
*=======================*
RandomForestClassifier
score: 0.9733333333333334
time: 0:00:00.704267
*=======================*
KNeighborsClassifier
score: 0.9866666666666667
time: 0:00:00.162659
*=======================*
LogisticRegression
score: 0.9733333333333334
time: 0:00:00.162601
*=======================*


#### Parametery najlepszego klasyfikatora

Sprawdzamy, który klasyfikator najlepiej wypadł i jakie miał parametry

In [16]:
classifiers_results_iris.sort(key=itemgetter('best_score'), reverse=True)  # sortujemy po najlepszym wyniku
print(classifiers_results_iris[0]['name'])
print(classifiers_results_iris[0]['grid_search'].best_params_)

KNeighborsClassifier
{'p': 2, 'n_neighbors': 5}


Ranking parametrów najlepszego klasyfikatora

In [17]:
report(classifiers_results_iris[0]['grid_search'].grid_scores_)

Model with rank: 1
Mean validation score: 0.98667 (std: 0.00924)
Parameters: {'p': 2, 'n_neighbors': 5}

Model with rank: 2
Mean validation score: 0.97333 (std: 0.00897)
Parameters: {'p': 1, 'n_neighbors': 5}

Model with rank: 3
Mean validation score: 0.97333 (std: 0.00897)
Parameters: {'p': 1, 'n_neighbors': 10}



### Dataset Digits

In [18]:
X, y = digits_dataset['data'], digits_dataset['target']
classifiers_results_digits = test_classifiers(classifiers_to_test, X , y)

SVC
score: 0.9515859766277128
time: 0:00:02.220477
*=======================*
LinearSVC
score: 0.9298831385642737
time: 0:00:10.328087
*=======================*
ExtraTreesClassifier
score: 0.9482470784641068
time: 0:00:00.833098
*=======================*
RandomForestClassifier
score: 0.9332220367278798
time: 0:00:01.177249
*=======================*
KNeighborsClassifier
score: 0.9627156371730662
time: 0:00:00.557140
*=======================*
LogisticRegression
score: 0.9298831385642737
time: 0:00:04.984364
*=======================*


#### Parametry najlepszego klasyfikatora

In [19]:
classifiers_results_digits.sort(key=itemgetter('best_score'), reverse=True)
print(classifiers_results_digits[0]['name'])
print(classifiers_results_digits[0]['grid_search'].best_params_)

KNeighborsClassifier
{'p': 2, 'n_neighbors': 5}


### Porównanie parametrów klasyfikatorów dla obu zestawów danych

Tworzymy **pandas.DataFrame** z wyników obu testów. Następnie łączymy je poziomo, dzięki czemu mamy dobre porównanie klasyfikatorów

In [20]:
iris_results_df = pd.DataFrame([[result['name'], result['best_score']] 
                                for result in classifiers_results_iris],
                               columns=['Iris', 'best_score'])
digits_results_df = pd.DataFrame([[result['name'], result['best_score']] 
                                  for result in classifiers_results_digits],
                                 columns=['Digits', 'best_score'])
pd.concat([iris_results_df, digits_results_df], axis=1)

,Iris,best_score,Digits,best_score
0,KNeighborsClassifier,0.986667,KNeighborsClassifier,0.962716
1,SVC,0.980000,SVC,0.951586
2,LinearSVC,0.973333,ExtraTreesClassifier,0.948247
3,RandomForestClassifier,0.973333,RandomForestClassifier,0.933222
4,LogisticRegression,0.973333,LinearSVC,0.929883
5,ExtraTreesClassifier,0.966667,LogisticRegression,0.929883


### Classification report

W przypadku klasyfikacji wielu kategorii warto sprawdzić, jaki jest wynik klasyfikacji każdej kategorii. W tym celu możemy skorzystać z gotowej metody **sklearn.metrics.classification_report(y_true, y_pred)**

In [21]:
X, y = digits_dataset['data'], digits_dataset['target']

**GridSearchCV** po wybraniu najlepszych parametrów klasyfikatora, przeucza go na całym zestawie danych i sam zaczyna działać jak klasyfikator.  
Tzn. Wywołanie metory **predict(X)** na obiekcie **GridSearchCV**, zwraca wynik metody **predict(X)** na wybranym (najlepszym) klasyfikatorze (GridSearchCV.best_estimator)

In [22]:
y_pred = classifiers_results_digits[0]['grid_search'].predict(X) # Robimy predykcje na najlepszym klasyfikatorze dla zestawu digits

W pierwszej kolumnie są klasy (możemy przekazać nazwy klas do fukcji **classification_report**).  
Kolumny **precision**, **recall**, **f1-score** zawierają wyniki dla każdej klasy.  
Kolumna **support** to liczność danej klasy

In [23]:
print(metrics.classification_report(y, y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       178
          1       0.98      1.00      0.99       182
          2       1.00      1.00      1.00       177
          3       0.98      0.99      0.99       183
          4       0.99      1.00      1.00       181
          5       0.99      0.99      0.99       182
          6       0.99      1.00      0.99       181
          7       0.99      0.99      0.99       179
          8       0.99      0.98      0.98       174
          9       0.99      0.96      0.97       180

avg / total       0.99      0.99      0.99      1797



## Przykładowe wykorzystanie wyuczonego klasyfikatora

In [24]:
classifier = classifiers_results_iris[0]['grid_search']  # Klasyfikator dla zestawu iris

In [25]:
iris_dataset['data'][0]

array([ 5.1,  3.5,  1.4,  0.2])

In [26]:
iris_dataset['target'][0]

0

In [27]:
classifier.predict([iris_dataset['data'][0]])

array([0])